In [3]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import plotly.graph_objects as go

# Read ICE arrests data from GitHub
url = "https://github.com/deportationdata/ice/raw/refs/heads/main/data/arrests-latest.xlsx"

# Download the file with requests
response = requests.get(url)
df = pd.read_excel(BytesIO(response.content))

# Display basic information about the data
print(f"Data shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
df.head()

Data shape: (291722, 23)

Column names:
['apprehension_date', 'apprehension_state', 'apprehension_aor', 'final_program', 'final_program_group', 'apprehension_method', 'apprehension_criminality', 'case_status', 'case_category', 'departed_date', 'departure_country', 'final_order_yes_no', 'final_order_date', 'birth_year', 'citizenship_country', 'gender', 'apprehension_site_landmark', 'unique_identifier', 'apprehension_date_time', 'duplicate_likely', 'file_original', 'sheet_original', 'row_original']

First few rows:


,apprehension_date,apprehension_state,apprehension_aor,final_program,final_program_group,apprehension_method,apprehension_criminality,case_status,case_category,departed_date,...,birth_year,citizenship_country,gender,apprehension_site_landmark,unique_identifier,apprehension_date_time,duplicate_likely,file_original,sheet_original,row_original
0,2023-09-01,CALIFORNIA,San Francisco Area of Responsibility,ERO Criminal Alien Program,ICE,CAP Federal Incarceration,1 Convicted Criminal,6-Deported/Removed - Deportability,[16] Reinstated Final Order,2023-09-02,...,1972,MEXICO,Male,"FRE GENERAL AREA, NON-SPECIFIC",3ddc9dfa23c14851a7cd709ad2e6c52a4e36a08b,2023-09-01 00:00:00,0.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,69540
1,2023-09-01,SOUTH CAROLINA,Atlanta Area of Responsibility,ERO Criminal Alien Program,ICE,CAP Local Incarceration,2 Pending Criminal Charges,8-Excluded/Removed - Inadmissibility,[8C] Excludable / Inadmissible - Administrativ...,2024-01-17,...,1994,HONDURAS,Male,"RICHLAND COUNTY, SC",8b3087d9852e9db2203541ebb2fc90826c74a647,2023-09-01 00:00:00,0.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,156756
2,2023-09-01,NaN,NaN,Alternatives to Detention,ICE,ERO Reprocessed Arrest,3 Other Immigration Violator,E-Charging Document Canceled by ICE,[8A] Excludable / Inadmissible - Hearing Not C...,NaT,...,1966,ECUADOR,Male,NaN,db5178743c5753e8acf6f2f2eca88af68395073f,2023-09-01 00:19:00,1.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,247560
3,2023-09-01,NaN,NaN,Alternatives to Detention,ICE,ERO Reprocessed Arrest,3 Other Immigration Violator,ACTIVE,[8D] Excludable / Inadmissible - Under Adjudic...,NaT,...,1966,ECUADOR,Male,NaN,db5178743c5753e8acf6f2f2eca88af68395073f,2023-09-01 00:19:00,1.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,247561
4,2023-09-01,NaN,Phoenix Area of Responsibility,Detained Docket Control,ICE,ERO Reprocessed Arrest,3 Other Immigration Violator,ACTIVE,[8B] Excludable / Inadmissible - Under Adjudic...,NaT,...,1999,INDIA,Female,NaN,d24c268740238eff4206067e9be369a2b872bc51,2023-09-01 01:09:44,0.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,237391


In [8]:
# Convert apprehension_date to datetime
df['apprehension_date'] = pd.to_datetime(df['apprehension_date'])

# Extract year and month
df['year_month'] = df['apprehension_date'].dt.to_period('M')

# Count arrests by month
monthly_arrests = df.groupby('year_month').size().reset_index(name='arrests')
monthly_arrests['year_month'] = monthly_arrests['year_month'].dt.to_timestamp()

# Convert to thousands
monthly_arrests['arrests_thousands'] = monthly_arrests['arrests'] / 1000

# Filter for the date range shown in the chart (Sept 2023 - July 2025)
monthly_arrests = monthly_arrests[
    (monthly_arrests['year_month'] >= '2023-09-01') & 
    (monthly_arrests['year_month'] <= '2025-07-31')
]

# Create the bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly_arrests['year_month'],
    y=monthly_arrests['arrests_thousands'],
    marker=dict(
        color='#E74C3C',  # Red color matching the image
        line=dict(width=0)
    ),
    hovertemplate='%{y:.1f}k arrests<extra></extra>'
))

# Add administration labels - positioned directly on the chart area
# Biden label positioned in the middle of his period
fig.add_annotation(
    x=8,  # Middle of Biden period (0 to 16 = 17 months, center at 8)
    y=28,
    text='Biden',
    showarrow=False,
    font=dict(size=18, color='black', family='Roboto, Arial, sans-serif'),
    xanchor='center',
    yanchor='middle'
)

# Trump label positioned in the middle of his period
fig.add_annotation(
    x=19.5,  # Middle of Trump period (17 to 22 = 6 months, center at 19.5)
    y=28,
    text='Trump',
    showarrow=False,
    font=dict(size=18, color='black', family='Roboto, Arial, sans-serif'),
    xanchor='center',
    yanchor='middle'
)

# Create categorical x-axis labels with years only at year boundaries
x_labels = []
x_categories = []
year_positions = []  # Track year label positions for centering

for i, date in enumerate(monthly_arrests['year_month']):
    month = date.month
    year = date.year
    month_letter = 'JFMAMJJASOND'[month - 1]
    
    # Create categorical label
    x_categories.append(f"{year}-{month:02d}")
    
    # Show only month letters for the tickmarks
    x_labels.append(month_letter)
    
    # Track positions where we want to add year labels
    if month == 1 or i == 0:
        year_positions.append((i, year))

# Update the trace to use categorical x-axis
fig.data[0].x = x_categories

# Update layout to match the image style
fig.update_layout(
    title=dict(
        text='Migrant arrests by Immigration and<br>Customs Enforcement (ICE), \'000',
        font=dict(size=24, color='black', family='Roboto, Arial, sans-serif', weight='bold'),
        x=0.02,
        xanchor='left',
        y=0.88,
        yanchor='top'
    ),
    xaxis=dict(
        type='category',
        tickmode='array',
        tickvals=x_categories,
        ticktext=x_labels,
        tickfont=dict(size=14, family='Roboto, Arial, sans-serif'),
        showgrid=False,
        showline=True,
        linecolor='black',
        linewidth=1,
        zeroline=False,
        ticks='',
        ticklen=0,
        range=[-0.5, len(x_categories) + 0.2]
    ),
    yaxis=dict(
        range=[0, 32],
        tickvals=[0, 5, 10, 15, 20, 25, 30],
        tickfont=dict(size=14, family='Roboto, Arial, sans-serif'),
        showgrid=True,
        gridcolor='#D0D0D0',
        gridwidth=1,
        showline=False,
        zeroline=False,
        title='',
        side='right',
        tickmode='array',
        layer='below traces',
        ticks='',
        ticklen=0,
        showticklabels=False
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=550,
    margin=dict(l=20, r=20, t=130, b=80),
    showlegend=False,
    bargap=0.3,
    hoverlabel=dict(
        bgcolor='black',
        font_size=14,
        font_family='Roboto, Arial, sans-serif'
    ),
    shapes=[
        # Red rectangle above the title
        dict(
            type='rect',
            xref='paper',
            yref='paper',
            x0=0,
            y0=1.28,
            x1=0.06,
            y1=1.31,
            fillcolor='#E74C3C',
            line=dict(width=0)
        ),
        # Thin vertical line at January 2025 column
        dict(
            type='line',
            xref='x',
            yref='y',
            x0=16,  # January 2025 position
            y0=0,
            x1=16,
            y1=30,
            line=dict(color='black', width=1),
            layer='below'
        )
    ]
)

# Add y-axis labels on top of gridlines
for tick_val in [0, 5, 10, 15, 20, 25, 30]:
    fig.add_annotation(
        x=1.0,
        xref='paper',
        y=tick_val,
        yref='y',
        text=str(tick_val),
        showarrow=False,
        font=dict(size=14, family='Roboto, Arial, sans-serif', color='black'),
        xanchor='right',
        yanchor='bottom'
    )

# Add centered year labels below the x-axis
for start_idx, year in year_positions:
    # Find the end of this year (or end of data)
    if year == 2023:
        # Sept 2023 to Dec 2023: 4 months, center at position start_idx + 1.5
        center_x = start_idx + 1.5
    elif year == 2024:
        # Jan 2024 to Dec 2024: 12 months, center at position start_idx + 5.5
        center_x = start_idx + 5.5
    elif year == 2025:
        # Jan 2025 to July 2025: 7 months, center at position start_idx + 3
        center_x = start_idx + 3
    
    fig.add_annotation(
        x=center_x,
        xref='x',
        y=-0.06,
        yref='paper',
        text=str(year),
        showarrow=False,
        font=dict(size=14, family='Roboto, Arial, sans-serif', color='black'),
        xanchor='center',
        yanchor='top'
    )

# Add tick marks between columns
# First tick mark (before first column) - regular length since it's at the edge
fig.add_shape(
    type='line',
    xref='x',
    yref='paper',
    x0=-0.5,
    y0=0,
    x1=-0.5,
    y1=-0.015,  # Regular tick
    line=dict(color='black', width=1)
)

# Tick marks between columns
for i in range(len(x_categories) - 1):
    # Check if this tick separates different years
    current_year = int(x_categories[i].split('-')[0])
    next_year = int(x_categories[i + 1].split('-')[0])
    is_year_boundary = (current_year != next_year)
    tick_length = -0.030 if is_year_boundary else -0.015
    
    fig.add_shape(
        type='line',
        xref='x',
        yref='paper',
        x0=i + 0.5,  # Between columns
        y0=0,
        x1=i + 0.5,
        y1=tick_length,
        line=dict(color='black', width=1)
    )

# Last tick mark (after last column) - regular length since it's at the edge
fig.add_shape(
    type='line',
    xref='x',
    yref='paper',
    x0=len(x_categories) - 0.5,
    y0=0,
    x1=len(x_categories) - 0.5,
    y1=-0.015,  # Regular length
    line=dict(color='black', width=1)
)

# Add source annotation
fig.add_annotation(
    text='Source: Deportation Data Project (Replication of chart from The Economist)',
    xref='paper',
    yref='paper',
    x=0,
    y=-0.18,
    showarrow=False,
    font=dict(size=13, color='#666666', family='Roboto, Arial, sans-serif'),
    xanchor='left'
)

fig.show()

In [9]:
# Save the chart as an HTML file with responsive sizing
fig.update_layout(
    autosize=True,
    width=None,
    height=None
)

fig.write_html(
    "../graphs/ice_migrant_arrests_economist.html",
    config={
        'responsive': True,
        'displayModeBar': True
    }
)
print("\nChart saved to ../graphs/ice_migrant_arrests_economist.html")


Chart saved to ../graphs/ice_migrant_arrests_economist.html


In [10]:
# NEW CHART: Biden months in blue, Trump months stacked (gray baseline + red excess)
# Calculate Biden average (Sept 2023 - Jan 2025 = 17 months)
biden_avg = monthly_arrests.iloc[:17]['arrests_thousands'].mean()

print(f"Biden average arrests (Sept 2023 - Jan 2025): {biden_avg:.2f}k")
print(f"\nTrump months breakdown:")

# Create separate traces for stacked bars
biden_data = monthly_arrests.iloc[:17].copy()
trump_data = monthly_arrests.iloc[17:].copy()

# For Trump months, split into gray baseline and red excess
trump_baseline = [biden_avg] * len(trump_data)
trump_excess = (trump_data['arrests_thousands'] - biden_avg).tolist()

for i, (idx, row) in enumerate(trump_data.iterrows()):
    print(f"{row['year_month'].strftime('%b %Y')}: Total={row['arrests_thousands']:.2f}k, Baseline={biden_avg:.2f}k, Excess={trump_excess[i]:.2f}k")

# Create the stacked bar chart
fig2 = go.Figure()

# Biden months - blue bars
fig2.add_trace(go.Bar(
    x=x_categories[:17],
    y=biden_data['arrests_thousands'],
    marker=dict(
        color='#4259AA',
        line=dict(width=0)
    ),
    name='Biden Period',
    hovertemplate='%{y:.1f}k arrests<extra></extra>'
))

# Trump months - gray baseline (bottom)
fig2.add_trace(go.Bar(
    x=x_categories[17:],
    y=trump_baseline,
    marker=dict(
        color='#D3D3D3',  # Light gray
        line=dict(width=0)
    ),
    name='Baseline (Avg. from Biden Period)',
    hovertemplate='%{y:.1f}k arrests<extra></extra>'
))

# Trump months - red excess (top)
fig2.add_trace(go.Bar(
    x=x_categories[17:],
    y=trump_excess,
    marker=dict(
        color='#E74C3C',
        line=dict(width=0)
    ),
    name='Increment in Trump Period',
    hovertemplate='%{y:.1f}k arrests<extra></extra>'
))

# Add administration labels
fig2.add_annotation(
    x=8,
    y=28,
    text='Biden',
    showarrow=False,
    font=dict(size=18, color='black', family='Roboto, Arial, sans-serif'),
    xanchor='center',
    yanchor='middle'
)

fig2.add_annotation(
    x=19.5,
    y=28,
    text='Trump',
    showarrow=False,
    font=dict(size=18, color='black', family='Roboto, Arial, sans-serif'),
    xanchor='center',
    yanchor='middle'
)

# Update layout
fig2.update_layout(
    title=dict(
        text='Migrant arrests by Immigration and<br>Customs Enforcement (ICE), \'000',
        font=dict(size=24, color='black', family='Roboto, Arial, sans-serif', weight='bold'),
        x=0.02,
        xanchor='left',
        y=0.88,
        yanchor='top'
    ),
    xaxis=dict(
        type='category',
        tickmode='array',
        tickvals=x_categories,
        ticktext=x_labels,
        tickfont=dict(size=14, family='Roboto, Arial, sans-serif'),
        showgrid=False,
        showline=True,
        linecolor='black',
        linewidth=1,
        zeroline=False,
        ticks='',
        ticklen=0,
        range=[-0.5, len(x_categories) + 0.2]
    ),
    yaxis=dict(
        range=[0, 32],
        tickvals=[0, 5, 10, 15, 20, 25, 30],
        tickfont=dict(size=14, family='Roboto, Arial, sans-serif'),
        showgrid=True,
        gridcolor='#D0D0D0',
        gridwidth=1,
        showline=False,
        zeroline=False,
        title='',
        side='right',
        tickmode='array',
        layer='below traces',
        ticks='',
        ticklen=0,
        showticklabels=False
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=550,
    margin=dict(l=20, r=20, t=130, b=80),
    showlegend=False,
    bargap=0.3,
    barmode='stack',  # Stack the Trump bars
    hoverlabel=dict(
        bgcolor='black',
        font_size=14,
        font_family='Roboto, Arial, sans-serif'
    ),
    shapes=[
        # Red rectangle above the title
        dict(
            type='rect',
            xref='paper',
            yref='paper',
            x0=0,
            y0=1.28,
            x1=0.06,
            y1=1.31,
            fillcolor='#E74C3C',
            line=dict(width=0)
        ),
        # Thin vertical line at January 2025 column
        dict(
            type='line',
            xref='x',
            yref='y',
            x0=16,
            y0=0,
            x1=16,
            y1=30,
            line=dict(color='black', width=1),
            layer='below'
        )
    ]
)

# Add y-axis labels on top of gridlines
for tick_val in [0, 5, 10, 15, 20, 25, 30]:
    fig2.add_annotation(
        x=1.0,
        xref='paper',
        y=tick_val,
        yref='y',
        text=str(tick_val),
        showarrow=False,
        font=dict(size=14, family='Roboto, Arial, sans-serif', color='black'),
        xanchor='right',
        yanchor='bottom'
    )

# Add centered year labels below the x-axis
for start_idx, year in year_positions:
    if year == 2023:
        center_x = start_idx + 1.5
    elif year == 2024:
        center_x = start_idx + 5.5
    elif year == 2025:
        center_x = start_idx + 3
    
    fig2.add_annotation(
        x=center_x,
        xref='x',
        y=-0.06,
        yref='paper',
        text=str(year),
        showarrow=False,
        font=dict(size=14, family='Roboto, Arial, sans-serif', color='black'),
        xanchor='center',
        yanchor='top'
    )

# Add tick marks between columns
fig2.add_shape(
    type='line',
    xref='x',
    yref='paper',
    x0=-0.5,
    y0=0,
    x1=-0.5,
    y1=-0.015,
    line=dict(color='black', width=1)
)

for i in range(len(x_categories) - 1):
    current_year = int(x_categories[i].split('-')[0])
    next_year = int(x_categories[i + 1].split('-')[0])
    is_year_boundary = (current_year != next_year)
    tick_length = -0.045 if is_year_boundary else -0.015
    
    fig2.add_shape(
        type='line',
        xref='x',
        yref='paper',
        x0=i + 0.5,
        y0=0,
        x1=i + 0.5,
        y1=tick_length,
        line=dict(color='black', width=1)
    )

fig2.add_shape(
    type='line',
    xref='x',
    yref='paper',
    x0=len(x_categories) - 0.5,
    y0=0,
    x1=len(x_categories) - 0.5,
    y1=-0.015,
    line=dict(color='black', width=1)
)

# Add legend above the chart and below the title
# Blue square for Biden period
fig2.add_shape(
    type='rect',
    xref='paper',
    yref='paper',
    x0=0.15,
    y0=1.00,
    x1=0.17,
    y1=1.02,
    fillcolor='#4259AA',
    line=dict(width=0)
)
fig2.add_annotation(
    text='Biden Period',
    xref='paper',
    yref='paper',
    x=0.18,
    y=1.01,
    showarrow=False,
    font=dict(size=12, color='black', family='Roboto, Arial, sans-serif'),
    xanchor='left',
    yanchor='middle'
)

# Light gray square for baseline
fig2.add_shape(
    type='rect',
    xref='paper',
    yref='paper',
    x0=0.37,
    y0=1.00,
    x1=0.39,
    y1=1.02,
    fillcolor='#D3D3D3',
    line=dict(width=0)
)
fig2.add_annotation(
    text='Baseline (Avg. from Biden Period)',
    xref='paper',
    yref='paper',
    x=0.40,
    y=1.01,
    showarrow=False,
    font=dict(size=12, color='black', family='Roboto, Arial, sans-serif'),
    xanchor='left',
    yanchor='middle'
)

# Red square for excess
fig2.add_shape(
    type='rect',
    xref='paper',
    yref='paper',
    x0=0.68,
    y0=1.00,
    x1=0.70,
    y1=1.02,
    fillcolor='#E74C3C',
    line=dict(width=0)
)
fig2.add_annotation(
    text='Increment in Trump Period',
    xref='paper',
    yref='paper',
    x=0.71,
    y=1.01,
    showarrow=False,
    font=dict(size=12, color='black', family='Roboto, Arial, sans-serif'),
    xanchor='left',
    yanchor='middle'
)

# Add source annotation
fig2.add_annotation(
    text='Source: Deportation Data Project (Modification of chart from The Economist)',
    xref='paper',
    yref='paper',
    x=0,
    y=-0.18,
    showarrow=False,
    font=dict(size=13, color='#666666', family='Roboto, Arial, sans-serif'),
    xanchor='left'
)

fig2.show()

Biden average arrests (Sept 2023 - Jan 2025): 9.48k

Trump months breakdown:
Feb 2025: Total=17.32k, Baseline=9.48k, Excess=7.84k
Mar 2025: Total=18.75k, Baseline=9.48k, Excess=9.27k
Apr 2025: Total=17.86k, Baseline=9.48k, Excess=8.38k
May 2025: Total=22.65k, Baseline=9.48k, Excess=13.17k
Jun 2025: Total=30.40k, Baseline=9.48k, Excess=20.92k
Jul 2025: Total=23.63k, Baseline=9.48k, Excess=14.15k


In [11]:
# Save the chart as an HTML file with responsive sizing
fig2.update_layout(
    autosize=True,
    width=None,
    height=None
)

fig2.write_html(
    "../graphs/ice_migrant_arrests_economist_modified.html",
    config={
        'responsive': True,
        'displayModeBar': True
    }
)
print("\nChart saved to ../graphs/ice_migrant_arrests_economist_modified.html")


Chart saved to ../graphs/ice_migrant_arrests_economist_modified.html
